# MRL ST2024 MC Questions

## Create MC Format 

In [7]:
from pathlib import Path
import pandas as pd
import numpy as np
import random
import plotly.express as px
BASE_PATH = Path('~/Documents/MRL/MC_exports')

char2idx = {'A': 0, 'B': 1, 'C': 2, 'D': 3}
idx2char = {0: 'A', 1: 'B', 2: 'C', 3: 'D'}

In [8]:
def shuffle_answers(answers):
    """shuffle the answers, keep true answer label

    Args:
        answers (list): 0th index is the correct answer, the rest are incorrect

    Returns:
        label (str): the correct answer label
        shuffled_answers (list): the shuffled answers
    """
    keys = ['A', 'B', 'C', 'D'] # A: correct, B-D: incorrect
    answer_choices = list(zip(keys, answers))
    rand_answers = random.sample(answer_choices, len(answer_choices))
    rand_answers = dict(rand_answers)
    labels = list(rand_answers.keys())

    label = labels.index('A') # Correct answer pos
    label = idx2char[label]

    return label, list(rand_answers.values())




In [9]:
langs = ['ALS', 'YO', 'TR', 'AZ', 'IG']
val_lens = [200, 200, 195, 200, 200] # remaining are for test

for i, lang in enumerate(langs):
    df = pd.read_csv(BASE_PATH / f'{lang}.csv')

    df = df[['annotation_id', 'text', 'question_updated', 'answer_updated', 'wrong_answer_1', 'wrong_answer_2', 'wrong_answer_3']]
    df.rename(columns={'question_updated': 'question', 'answer_updated': 'A', 'wrong_answer_1': 'B', 'wrong_answer_2': 'C', 'wrong_answer_3': 'D'}, inplace=True)
    df = df.loc[val_lens[i]:]
    
    df['label'] = 'A'
    for index, row in df.iterrows():
        answers = [row['A'], row['B'], row['C'], row['D']]
        label, answers_shuffled = shuffle_answers(answers)
        
        assert answers[0] == answers_shuffled[char2idx[label]], f'issue with shuffling: \ngold label: {label}, \ngold answer: {answers[0]} \nAnswers shuffled: {answers_shuffled}'
        
        df.at[index, 'label'] = label
        df.at[index, 'A'] = answers_shuffled[0]
        df.at[index, 'B'] = answers_shuffled[1]
        df.at[index, 'C'] = answers_shuffled[2]
        df.at[index, 'D'] = answers_shuffled[3]

    
    df.to_csv(BASE_PATH / f'MC_{lang}_reference_test.csv', index=False)
    
    df_unlabeled = df.drop(columns=['label'])
    df_unlabeled['prediction'] = ''
    df_unlabeled.to_csv(BASE_PATH / f'MC_{lang}_test.predict', index=False)

    print(f'{lang}: {len(df)}')
    
    
    hist = px.histogram(df, x="label", nbins=4)
    
    
    
    

hist.show()


ALS: 650
YO: 659
TR: 0
AZ: 153
IG: 747


## Evaluate MC Answers

In [10]:
y_true = df['label'].to_list()
answer_options = ['A', 'B', 'C', 'D']
y_pred = random.choices(answer_options, k=len(y_true))

len(y_true), len(y_pred)


(747, 747)

In [11]:
import sklearn.metrics as metrics

accuracy = metrics.accuracy_score(y_true, y_pred, normalize=True)
f1 = metrics.f1_score(y_true, y_pred, average='micro')
precision = metrics.precision_score(y_true, y_pred, average='micro')
recall = metrics.recall_score(y_true, y_pred, average='micro')

recall

0.2222222222222222